# Using enterprise to analyze PTA data

In this notebook you will learn:
* How to use `enterprise` to interact with IPTA data,
* How to setup an analysis of indiviudual pulsar noise properties,
* How to search in PTA data for GWs,
* How to perform Bayesian model selection,
* How to post-process your results.

# Pre-requisites (installation etc.)

- **Install `miniconda` locally**

    `wget -q https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh`
    
    `bash Miniconda2-latest-Linux-x86_64.sh -b -p ~/.local/opt/miniconda2`
    
    `rm Miniconda2-latest-Linux-x86_64.sh`


- **Add miniconda’s `python`  to the front of your `$PATH`**

    `echo "export PATH=$HOME/.local/opt/miniconda2/bin:$PATH" >> .bashrc`
    
    `source .bashrc`


- **Install the basic python packages**

    `conda install -y numpy==1.13.3 cython scipy`


- **Install latest `libstempo` from GitHub with `pip`.  `tempo2` should be installed automatically.  Add extra ephemerides if needed**

    `pip install git+https://github.com/vallis/libstempo@master`


- **Install more python packages**

    `conda install -y matplotlib ipython h5py mpi4py numexpr statsmodels astropy ephem`


- **Install non-conda packages with `pip`**

    `pip install healpy acor line_profiler jplephem corner numdifftools`


- [optional] **Install `scikit-sparse`**

    `conda install -c menpo scikit-sparse`
   
   
- [optional] **Alternatively install `suite sparse` and then use pip to install `scikit-sparse` (maybe needed with python 3.6)**

    `conda install -c conda-forge suitesparse`
    
    `pip install git+https://github.com/scikit-sparse/scikit-sparse.git@master`


- **Install PTMCMC for sampling**

    `pip install git+https://github.com/jellis18/PTMCMCSampler@master`


- **Finally, install enterprise**

    `pip install git+https://github.com/nanograv/enterprise@master`

# Load modules

In [ ]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division

import numpy as np
import os, glob, json 
import matplotlib.pyplot as plt
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

# Define a python dictionary of pulsar names and PTAs

In [ ]:
# The pulsars we'll be analyzing
psrdict = {'J1713+0747': [{'pta': ['NANOGrav', 'PPTA']}], 
           'J1909-3744': [{'pta': ['NANOGrav', 'PPTA']}], 
           'J1640+2224': [{'pta': ['NANOGrav']}], 
           'J1600-3053': [{'pta': ['NANOGrav']}],
           'J2317+1439': [{'pta': ['NANOGrav']}], 
           'J1918-0642': [{'pta': ['NANOGrav']}], 
           'J1614-2230': [{'pta': ['NANOGrav']}], 
           'J1744-1134': [{'pta': ['NANOGrav', 'PPTA']}],
           'J0030+0451': [{'pta': ['NANOGrav']}], 
           'J2145-0750': [{'pta': ['NANOGrav']}], 
           'J1857+0943': [{'pta': ['NANOGrav']}], 
           'J1853+1303': [{'pta': ['NANOGrav']}], 
           'J0613-0200': [{'pta': ['NANOGrav']}],
           'J1455-3330': [{'pta': ['NANOGrav']}], 
           'J1741+1351': [{'pta': ['NANOGrav']}], 
           'J2010-1323': [{'pta': ['NANOGrav']}], 
           'J1024-0719': [{'pta': ['NANOGrav']}], 
           'J1012+5307': [{'pta': ['NANOGrav']}],
           'J0437-4715': [{'pta': ['PPTA']}]
          }
psrlist = psrdict.keys()

In [ ]:
psrlist

## Get par, tim, and noise files
Here we collect the tim and par files as well as noise files made from the `PAL2` code. These are the same par, tim, and noise files used in the 9-year analysis papers. We use the convienience function above to convert from `PAL2` noise files to `enterprise` parameter dictionaries.

In [ ]:
datadir = './partim_filtered_ppta_ng/'

In [ ]:
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

# filter
parfiles = [x for x in parfiles if x.split('/')[-1].split('.')[0] in psrlist]
timfiles = [x for x in timfiles if x.split('/')[-1].split('.')[0] in psrlist]

In [ ]:
len(parfiles)

## Load into Pulsar class list

* The `enterprise` Pulsar class uses `libstempo` to read in `par` and `tim` files, then stores all pulsar data into a `Pulsar` object. This object contains all data and meta-data needed for the ensuing pulsar and PTA analysis. You no longer to reference the `par` and `tim` files after this cell.
* Note below that you can explicitly declare which version of the JPL solar-system ephemeris model that will be used to compute the Roemer delay between the geocenter and the barycenter (e.g. `DE436`). Otherwise the default values will be taken from the `par` files. Explicitly declaring the version here is good practice.
* You can also explicitly set the clock file to a version of `BIPM`, e.g. `BIPM(2015)`. This is less important, and you can let the code take the value from the `par` file.
* When you execute the following cell, you will get warnings like `WARNING: Could not find pulsar distance for PSR ...`. Don't worry! This is expected, and fine. Not all pulsars have well constrained distances, and will be set to `1 kpc` with a `20%` uncertainty.

In [ ]:
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem='DE436', clk=None)
    psrs.append(psr)

# Single pulsar analysis

* `enterprise` is structured so that one first creates `parameters`, then `signals` that these `parameters` belong to, then finally a `model` that is the union of all `signals` and the `data`.

* We will show this explciitly below, then introduce some model shortcut code that will make your life easier.
* We test on `J1713+0747`.

In [ ]:
psr = [p for p in psrs if p.name == 'J1713+0747'][0]

In [ ]:
# find the maximum time span to set red-noise/DM-variation frequency sampling
tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = np.max(tmax) - np.min(tmin)

In [ ]:
# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# special selection for ECORR only use wideband NANOGrav data
selection2 = selections.Selection(selections.nanograv_backends)

## Create parameters

In [ ]:
# white noise parameters
#efac = parameter.Uniform(0.01, 10.0)
#equad = parameter.Uniform(-8.5, -5)
#ecorr = parameter.Uniform(-8.5, -5)
efac = parameter.Constant() 
equad = parameter.Constant() 
ecorr = parameter.Constant() 

# red noise parameters
log10_A = parameter.Uniform(-20, -11)
gamma = parameter.Uniform(0, 7)

# dm-variation parameters
log10_A_dm = parameter.Uniform(-20, -11)
gamma_dm = parameter.Uniform(0, 7)

In [ ]:
# [Optional] If fixing white-noise, read-in previously computed noisefiles
noisefiles = sorted(glob.glob('./partim_filtered_ppta_ng/noisefiles_ppta_ng_normal/*.json'))

params = {}
for nf in noisefiles:
    with open(nf, 'r') as fin:
        params.update(json.load(fin))

## Create signals

In [ ]:
# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection2)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

# DM-variations (powerlaw with 30 frequencies)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=30, Tspan=Tspan)
dm_pl = utils.powerlaw(log10_A=log10_A_dm, gamma=gamma_dm)
dm_gp = gp_signals.BasisGP(dm_pl, dm_basis, name='dm_gp')

# timing model
tm = gp_signals.TimingModel(use_svd=True)

## Piece the full model together

In [ ]:
# full model
s = ef + eq + rn + dm_gp + tm

In [ ]:
# intialize a single-pulsar pta model
models = []
   
if 'NANOGrav' in psr.flags['pta']:
    s2 = s + ec # ecorr only applied to NANOGrav data
    models.append(s2(psr))
else:
    models.append(s(psr))
    
pta = signal_base.PTA(models)

In [ ]:
# [Optional] Set white-noise parmeters from previous analysis
pta.set_default_params(params)

In [ ]:
pta.params

## Draw initial sample from model parameter space

In [ ]:
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

## Setup sampler (simple, with no tricks)

In [ ]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution

# where chains will be written to
outdir = './chains/ipta_dr2_ng_ppta_{}/'.format(str(psr.name))

# sampler object
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,
                 outDir=outdir, 
                 resume=False)

## Sample the parameter space

In [ ]:
# sampler for N steps
N = int(1e6)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## Simple post-processing

In [ ]:
chain = np.loadtxt(outdir + 'chain_1.txt')
burn = int(0.25 * chain.shape[0])

In [ ]:
# Find column of chain file corresponding to a parameter
ind = list(pta.param_names).index('J1713+0747_log10_A')

In [ ]:
# Make trace-plot to diagnose sampling
plt.plot(chain[burn:, ind])

In [ ]:
# Plot a histogram of the marginalized posterior distribution
plt.hist(chain[burn:,ind], 50, normed=True, histtype='stepfilled', 
         lw=2, color='C0', alpha=0.5);
plt.xlabel('J1713+0747_log10_A')
plt.ylabel('PDF')

In [ ]:
# Make 2d histogram plot
ind_redA = list(pta.param_names).index('J1713+0747_log10_A')
ind_redgam = list(pta.param_names).index('J1713+0747_gamma')
fig = corner.corner(chain[burn:, [ind_redA, ind_redgam]], 
                    labels=['J1713+0747_log10_A', 'J1713+0747_gamma'],
                   levels=[0.68,0.95]);

## Add a new custom function

In this example, we add a new custom function that takes the form of a dispersive ($\nu^{-2}$) exponential dip in the residuals of `J1713+0747` due to a void in the ISM plasma. This is a real effect that has been observed.

In [ ]:
@signal_base.function
def chrom_exp_decay(toas, freqs, log10_Amp=-7,
                    t0=54000, log10_tau=1.7, idx=2):
    """
    Chromatic exponential-dip delay term in TOAs.

    :param t0: time of exponential minimum [MJD]
    :param tau: 1/e time of exponential [s]
    :param log10_Amp: amplitude of dip
    :param idx: index of chromatic dependence

    :return wf: delay time-series [s]
    """
    t0 *= const.day
    tau = 10**log10_tau * const.day
    wf = -10**log10_Amp * np.heaviside(toas - t0, 1) * \
        np.exp(- (toas - t0) / tau)

    return wf * (1400 / freqs) ** idx

In [ ]:
def dm_exponential_dip(tmin, tmax, idx=2, name='dmexp'):
    """
    Returns chromatic exponential dip (i.e. TOA advance):

    :param tmin, tmax:
        search window for exponential dip time.
    :param idx:
        index of radio frequency dependence (i.e. DM is 2). If this is set
        to 'vary' then the index will vary from 1 - 6
    :param name: Name of signal

    :return dmexp:
        chromatic exponential dip waveform.
    """
    t0_dmexp = parameter.Uniform(tmin,tmax)
    log10_Amp_dmexp = parameter.Uniform(-10, -2)
    log10_tau_dmexp = parameter.Uniform(np.log10(5), np.log10(100))
    wf = chrom_exp_decay(log10_Amp=log10_Amp_dmexp, t0=t0_dmexp,
                         log10_tau=log10_tau_dmexp, idx=idx)
    dmexp = deterministic_signals.Deterministic(wf, name=name)

    return dmexp

In [ ]:
# full model
s = ef + eq + rn + dm_gp + tm

# intialize a single-pulsar pta model
models = []
    
if psr.name == 'J1713+0747' and 'NANOGrav' in psr.flags['pta']:
    s_prime = s + dm_exponential_dip(tmin=(tmin/86400.0), tmax=(tmax/86400.0)) + ec
    models.append(s_prime(psr))
elif psr.name == 'J1713+0747':
    s_prime = s + dm_exponential_dip(tmin=(tmin/86400.0), tmax=(tmax/86400.0))
    models.append(s_prime(psr))
elif 'NANOGrav' in psr.flags['pta']:
    s_prime = s + ec
    models.append(s_prime(psr))
else:
    models.append(s(psr))
    
pta = signal_base.PTA(models)

In [ ]:
# [Optional] Set white-noise parmeters from previous analysis
pta.set_default_params(params)

In [ ]:
pta.params

In [ ]:
# Sample as before
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution

# where chains will be written to
outdir = './chains/ipta_dr2_ng_ppta_{}_dmexp/'.format(str(psr.name))

# sampler object
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,
                 outDir=outdir, 
                 resume=False)

# sampler for N steps
N = int(1e6)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

In [ ]:
chain = np.loadtxt(outdir + 'chain_1.txt')
burn = int(0.5 * chain.shape[0])

In [ ]:
# Plot a histogram of the marginalized posterior distribution

ind = list(pta.param_names).index('J1713+0747_dmexp_log10_tau')

plt.hist(chain[burn:,ind], 50, normed=True, histtype='stepfilled', 
         lw=2, color='C0', alpha=0.5);
plt.xlabel('J1713+0747_dmexp_t0')
plt.ylabel('PDF')

## Now, the easy way to do all of this

Many of us have created shortcuts to carry out these tasks. You will find them in `enterprise_extensions`: https://github.com/stevertaylor/enterprise_extensions. Clone this repo, go into the cloned repo directory, then execute `pip install .`

This will install the package on your local machine.

In [ ]:
import enterprise_extensions
from enterprise_extensions import models, model_utils

In [ ]:
# Create a single pulsar model
pta = models.model_singlepsr_noise(psr, psd='powerlaw', 
                                   noisedict=params, 
                                   white_vary=False,
                                   dm_var=True, 
                                   dm_psd='powerlaw', dm_annual=True)

In [ ]:
# Setup a sampler instance.
# This will add some fanicer stuff than before, like prior draws, 
# and custom sample groupings.
sampler = model_utils.setup_sampler(pta, outdir=outdir, resume=False)

In [ ]:
# sampler for N steps
N = int(1e6)
x0 = x0 = np.hstack(p.sample() for p in pta.params)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

In [ ]:
chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
pp.plot_trace()

# Single-pulsar Model Selection

We want to be able to create custom noise descriptions per pulsar. Some pulsars will have funky noise features (like the dispersive dip in `J1713+0747`) while others will have more vanilla noise models. The sure-fire way to exlore this is through model selection. We want to incorporate how well a model describes the data, and the Occam penalty given through too many unneccessary parameters. A simple way to do this is given below.

## Create list of pta models for our model selection

In [ ]:
nmodels = 2
mod_index = np.arange(nmodels)

# Make dictionary of single-pulsar PTAs.
# Select between a power-law or t-process red-noise spectrum
pta = dict.fromkeys(mod_index)
pta[0] = models.model_singlepsr_noise(psr, psd='powerlaw', noisedict=params, 
                                      white_vary=False,
                                      dm_var=True, dm_psd='powerlaw', dm_annual=True)
pta[1] = models.model_singlepsr_noise(psr, psd='tprocess', noisedict=params, 
                                      white_vary=False,
                                      dm_var=True, dm_psd='powerlaw', dm_annual=True)

## Instanciate a collection of models

In [ ]:
super_model = model_utils.HyperModel(pta)

## Sample

In [ ]:
sampler = super_model.setup_sampler(resume=False, outdir=outdir)

In [ ]:
# sampler for N steps
N = int(5e6)
x0 = super_model.initial_sample()

In [ ]:
# sample
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## Post-process

In [ ]:
chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
# Histogram of model-indexing variable
plt.hist(chain[burn:,-5],);

## Baysian Odds-ratio

The odds-ratio is given by the relative number of sampling iterations spent in each sub-likelihood.

In [ ]:
print model_utils.odds_ratio(chain[burn:,-5], models=[0,1])